In [4]:
# 지방공항 국제선/
url = 'https://www.airport.kr/co/ko/cpr/statisticOfLocalAirport.do'

In [62]:
#각종 패키지 설치

import pandas as pd
import numpy as np

!pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

from urllib.request import urlopen
#from bs4 import BeautifulSoup
#from html_table_parser import parser_functions as parser
from selenium.webdriver.common.by import By

In [269]:
domestic_air_info = pd.DataFrame(columns = {'공항','운항_도착','운항_출발','운항_합계','여객_도착','여객_출발','여객_합계','화물_도착','화물_출발','화물_합계'})
                                                                                        
domestic_air_info.columns = [['공항','운항_도착','운항_출발','운항_합계',
                              '여객_도착','여객_출발','여객_합계',
                              '화물_도착','화물_출발','화물_합계']]



# 항공기 선택창 Xpath
xpath_airplane = '//*[@id="ARPLN_SE"]'
# 여객 선택창 (유임 승객)
xpath_type = '//*[@id="PAS_SEL"]'
# 노선 선택창
xpath_inter = '//*[@id="ROUTE_SE"]'
# 검색 버튼
xpath_search = '//*[@id="btnSearch"]'
# 시작 년도
xpath_start_year = '//*[@id="S_YEAR"]'
# 시작 월
xpath_start_month = '//*[@id="S_MONTH"]'
# 기준년도
xpath_end_year = '//*[@id="E_YEAR"]'
# 기준월
xpath_end_month = '//*[@id="E_MONTH"]'



# 웹드라이버 열기
driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")
driver.get(url)

# 항공기(여객기) 선택
driver.find_element(by=By.XPATH, value = xpath_airplane).click()
driver.find_element(by=By.XPATH, value = xpath_airplane).send_keys(Keys.DOWN)
driver.find_element(by=By.XPATH, value = xpath_airplane).send_keys(Keys.RETURN)

# 노선 선택 (국제선)
driver.find_element(by=By.XPATH, value = xpath_inter).click()
driver.find_element(by=By.XPATH, value = xpath_inter).send_keys(Keys.DOWN)
driver.find_element(by=By.XPATH, value = xpath_inter).send_keys(Keys.RETURN)

# 여객 선택 (유임 승객)
driver.find_element(by=By.XPATH, value = xpath_type).click()
driver.find_element(by=By.XPATH, value = xpath_type).send_keys(Keys.DOWN)
driver.find_element(by=By.XPATH, value = xpath_type).send_keys(Keys.RETURN)


for i in range(2013, 2022) : #13년도부터 21년도까지의 데이터 수집을 위해 반복문 설정
    driver.find_element(by=By.XPATH, value = xpath_start_year).send_keys(i) #시작년 체크
    driver.find_element(by=By.XPATH, value = xpath_end_year).send_keys(i) #종료년 체크
    for j in range(1,13) :
        if i == 2021 and j == 4 :
            break
        if j < 10 :
            jj = "0" + str(j)
        else :
            jj = str(j)
        driver.find_element(by=By.XPATH, value = xpath_start_month).send_keys(jj) #시작월 체크
        driver.find_element(by=By.XPATH, value = xpath_end_month).send_keys(jj) #종료월 체크
        driver.find_element(by=By.XPATH, value = xpath_search).click() # 검색 버튼 클릭
        time.sleep(0.1)
        
        df = driver.find_element(by=By.XPATH, value = '/html/body/section[2]/div/div/article[1]/table/tbody').text.split("\n")[1:-1]
        #"".join(driver.find_element(by=By.XPATH, value = '/html/body/section[2]/div/div/article[1]/table/tbody').text).replace('\n',' {} {}\n'.format(i,j)).split("\n")[:-1]
        a = pd.DataFrame(df)
        a.columns = ['col'] # 임시 컬럼명 설정
        a = a['col'].str.split(" ", expand=True)#.rename(columns={0:'공항', 1:"운항_도착",''})
        #pd.DataFrame(a.col.str.split(" ",9).tolist())
        a.columns = [['공항','운항_도착','운항_출발','운항_합계',
                              '여객_도착','여객_출발','여객_합계',
                              '화물_도착','화물_출발','화물_합계']
                             ]
        a['년'] = str(i)
        a['월'] = str(j)
        domestic_air_info = pd.concat([domestic_air_info,a],axis=0)

C:\Users\USER\AppData\Local\Temp/ipykernel_10516/4131572444.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")


In [277]:
da = domestic_air_info.copy()

In [278]:
da = da[['공항','운항_도착','운항_출발','운항_합계',
                              '여객_도착','여객_출발','여객_합계',
                              '화물_도착','화물_출발','화물_합계','년','월']
                             ]

In [279]:
da

,공항,운항_도착,운항_출발,운항_합계,여객_도착,여객_출발,여객_합계,화물_도착,화물_출발,화물_합계,년,월
0,김포,927,928,"1,855","152,431","161,486","313,917","2,487","3,657","6,144",2013,1
1,김해,"1,567","1,569","3,136","192,377","196,129","388,506","2,701","3,014","5,715",2013,1
2,제주,288,287,575,"31,607","32,066","63,673",219,429,648,2013,1
3,대구,47,47,94,"6,542","6,762","13,304",74,70,144,2013,1
4,무안,40,39,79,"4,716","4,395","9,111",65,48,113,2013,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1,제주,0,6,6,0,239,239,0,5,5,2021,2
2,대구,0,4,4,0,638,638,0,11,11,2021,2
0,김해,4,4,8,659,583,"1,242",13,10,23,2021,3
1,제주,0,8,8,0,275,275,0,5,5,2021,3


In [281]:
da.to_excel("국내공항 통계.xlsx")